In [1]:
import os
import sys
import json
import stanza
from tqdm import tqdm_notebook as tqdm

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

'/media/moju/data/work/ser'

In [2]:
sys.path.append(w_dir+'/ser_annotate')

In [3]:
from ser_annotate.std import *
from ser_annotate import config
from ser_annotate.dataset_reader.io_cano import CanoDict, CrpCano

2020-05-29 18:35:38 INFO: "zh" is an alias for "zh-hans"
2020-05-29 18:35:38 INFO: Loading these models for language: zh-hans (Simplified_Chinese):
| Processor | Package   |
-------------------------
| tokenize  | gsdsimp   |
| ner       | ontonotes |

2020-05-29 18:35:38 INFO: Use device: cpu
2020-05-29 18:35:38 INFO: Loading: tokenize
2020-05-29 18:35:38 INFO: Loading: ner
2020-05-29 18:35:38 INFO: Done loading processors!


In [4]:
test_file = config.SSQA_ANNOT / "PubB-G6a-0302.json1"

In [5]:
annots = jseq_load(test_file)

In [6]:
len(annots)

99

In [7]:
myCanos = CrpCano()

In [8]:
myCanos.load_annot_ssqa(annots)

In [9]:
len(myCanos)

99

In [10]:
test_q = myCanos['PubB-G6a-0302.jsonl-06-002408']

In [11]:
print(test_q)

qid:PubB-G6a-0302.jsonl-06-002408
qtext:「文文民調中心」問卷調查目前臺灣的家庭型態，發現「大家庭」是目前臺灣社會主要的家庭結構型態。
c_text:第三單元社會的變遷|||家庭兩性新關係|||＜家庭型態的轉變＞|||工業化不但改變了我們的生活方式，||連帶的也影響家庭的型態。|||首先是居住型態的改變。||以前農業社會中的大家庭，||現在已經很少見，||取而代之的是三代同堂的折衷家庭，||和一對夫婦與未婚子女所組成的小家庭。|||近年來，||由於離婚或意外事故等因素，||產生了許多單親家庭；||而父母到都市就業，||將小孩託付給祖父母照顧的隔代教養家庭也時有所見；||而因異國婚姻所組成的家庭也有增加的趨勢。||這些家庭在經濟和子女教養上，||需要社會更多的關注，||因此，||政府應積極訂定完善的社會福利制度，||設立足夠的托兒機構，||協助家庭解決年幼子女的教養問題。|||夫婦與未婚子女所組成的小家庭。|||由祖父母、||父母與子女所組成三代同堂的折衷家庭。|||大家庭是以往農業社會常見的家庭形態。|||民國50～99年臺閩地區每戶家庭平均人口數變遷圖。|||社區重視單親暨隔代教養家庭子女課業問題，||特別舉辦輔導說明會。|||嘉義縣東石鄉成立新移民關懷據點，||協助新移民適應環境和生活。|||考量到新移民的交通便利問題，||一些縣市政府便設立新移民機車考照班，||協助新移民考取駕照。|||兒童福利服務中心舉辦「單親家庭親子捏陶創作」活動。|||＊觀點小視窗：||隔代教養家庭由祖父母與孫子女所組成的家庭。|||＜性別平等向前走＞|||隨著女性教育程度的提高，||以及性別平等觀念的提倡，||女性就業人口快速增加，||在各行各業都有傑出的表現。||尤其在傳統上屬於男性的專業工作，||也有愈來愈多的女性投入。|||但是，||傳統「男主外、||女主內」的，||仍造成部分女性很大的困擾。||在講求性別平等的今天，||女性外出就業已經是社會的趨勢。||在家庭中，||除了男女需要互相尊重外，||更應該將家務視為全家一起分擔的責任。|||近年來，||臺灣地區女性從事專業工作的比例，||有增加的趨勢；||而靠體力的女性工作者，||則逐漸減少。|||女性就業人口已成為推動臺灣社會發展的重要力量，||而傳統中屬於女性的專業工作也有愈來愈多的男性投入

In [12]:
test_q.c_text[88:91]

'大家庭'

In [13]:
from ser_annotate.dataset_reader.io_SSQA import str_lesson2parags, str_lesson2st

In [14]:
str_lesson2parags(test_q.c_text)

['第三單元社會的變遷',
 '家庭兩性新關係',
 '＜家庭型態的轉變＞',
 '工業化不但改變了我們的生活方式，連帶的也影響家庭的型態。',
 '首先是居住型態的改變。以前農業社會中的大家庭，現在已經很少見，取而代之的是三代同堂的折衷家庭，和一對夫婦與未婚子女所組成的小家庭。',
 '近年來，由於離婚或意外事故等因素，產生了許多單親家庭；而父母到都市就業，將小孩託付給祖父母照顧的隔代教養家庭也時有所見；而因異國婚姻所組成的家庭也有增加的趨勢。這些家庭在經濟和子女教養上，需要社會更多的關注，因此，政府應積極訂定完善的社會福利制度，設立足夠的托兒機構，協助家庭解決年幼子女的教養問題。',
 '夫婦與未婚子女所組成的小家庭。',
 '由祖父母、父母與子女所組成三代同堂的折衷家庭。',
 '大家庭是以往農業社會常見的家庭形態。',
 '民國50～99年臺閩地區每戶家庭平均人口數變遷圖。',
 '社區重視單親暨隔代教養家庭子女課業問題，特別舉辦輔導說明會。',
 '嘉義縣東石鄉成立新移民關懷據點，協助新移民適應環境和生活。',
 '考量到新移民的交通便利問題，一些縣市政府便設立新移民機車考照班，協助新移民考取駕照。',
 '兒童福利服務中心舉辦「單親家庭親子捏陶創作」活動。',
 '＊觀點小視窗：隔代教養家庭由祖父母與孫子女所組成的家庭。',
 '＜性別平等向前走＞',
 '隨著女性教育程度的提高，以及性別平等觀念的提倡，女性就業人口快速增加，在各行各業都有傑出的表現。尤其在傳統上屬於男性的專業工作，也有愈來愈多的女性投入。',
 '但是，傳統「男主外、女主內」的，仍造成部分女性很大的困擾。在講求性別平等的今天，女性外出就業已經是社會的趨勢。在家庭中，除了男女需要互相尊重外，更應該將家務視為全家一起分擔的責任。',
 '近年來，臺灣地區女性從事專業工作的比例，有增加的趨勢；而靠體力的女性工作者，則逐漸減少。',
 '女性就業人口已成為推動臺灣社會發展的重要力量，而傳統中屬於女性的專業工作也有愈來愈多的男性投入。',
 '新好男人巧扮「袋鼠男人」，體會孕婦在職場打拼的辛苦。',
 '政府制定家庭暴力防治法及設置「113全國婦幼保護專線」，以保護婦女與兒童的安全。',
 '夫妻參與產前講座，共同迎接家中新成員。',
 '性別工作平等法的實施，保障婦女在工作上的權益。',
 '

In [15]:
str_lesson2st(test_q.c_text)

[[0, 9, [[0, 9, '第三單元社會的變遷']]],
 [12, 19, [[12, 19, '家庭兩性新關係']]],
 [22, 31, [[22, 31, '＜家庭型態的轉變＞']]],
 [34, 64, [[34, 50, '工業化不但改變了我們的生活方式，'], [52, 64, '連帶的也影響家庭的型態。']]],
 [67,
  140,
  [[67, 78, '首先是居住型態的改變。'],
   [80, 92, '以前農業社會中的大家庭，'],
   [94, 102, '現在已經很少見，'],
   [104, 120, '取而代之的是三代同堂的折衷家庭，'],
   [122, 140, '和一對夫婦與未婚子女所組成的小家庭。']]],
 [143,
  315,
  [[143, 147, '近年來，'],
   [149, 162, '由於離婚或意外事故等因素，'],
   [164, 174, '產生了許多單親家庭；'],
   [176, 185, '而父母到都市就業，'],
   [187, 211, '將小孩託付給祖父母照顧的隔代教養家庭也時有所見；'],
   [213, 233, '而因異國婚姻所組成的家庭也有增加的趨勢。'],
   [235, 249, '這些家庭在經濟和子女教養上，'],
   [251, 261, '需要社會更多的關注，'],
   [263, 266, '因此，'],
   [268, 285, '政府應積極訂定完善的社會福利制度，'],
   [287, 297, '設立足夠的托兒機構，'],
   [299, 315, '協助家庭解決年幼子女的教養問題。']]],
 [318, 333, [[318, 333, '夫婦與未婚子女所組成的小家庭。']]],
 [336, 361, [[336, 341, '由祖父母、'], [343, 361, '父母與子女所組成三代同堂的折衷家庭。']]],
 [364, 382, [[364, 382, '大家庭是以往農業社會常見的家庭形態。']]],
 [385, 410, [[385, 410, '民國50～99年臺閩地區每戶家庭平均人口數變遷圖。']]],
 [413, 445, [[413, 433, '社區重視單親暨隔代教養家庭子女課業問題